# 1. Fuentes de datos externas

## 1.1 Importación

In [1]:
import pandas as pd
import numpy as np

### Dataset de estaciones metereologicas 

Obtenido de National Centers for Environmental Information (.gov)

[Enlace directo de dscarga](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwin1fTL1viRAxUBQzABHbosArgQFnoECAwQAQ&url=https%3A%2F%2Fwww.ncei.noaa.gov%2Fpub%2Fdata%2Fnoaa%2Fisd-history.csv&usg=AOvVaw3K8oxIQwTSdqoixdzTGvA3&opi=89978449)

In [2]:
df_stationMeteo = pd.read_csv(r'isd-history.csv')

In [3]:
df_stationMeteo.head(2)

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
0,007018,99999,WXPOD 7018,NaN,NaN,NaN,0.0,0.0,7018.0,20110309,20130730
1,007026,99999,WXPOD 7026,AF,NaN,NaN,0.0,0.0,7026.0,20120713,20170822


In [4]:
df_stationMeteo.columns

Index(['USAF', 'WBAN', 'STATION NAME', 'CTRY', 'STATE', 'ICAO', 'LAT', 'LON',
       'ELEV(M)', 'BEGIN', 'END'],
      dtype='object')

### Dataset de lon/lat de cada aeropuerto de USA

[Enlace directo de descarga](https://davidmegginson.github.io/ourairports-data/airports.csv)

In [5]:
df_airportUbicat = pd.read_csv(r'airports.csv') 

In [6]:
df_airportUbicat.head(2)

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,icao_code,iata_code,gps_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total RF Heliport,40.070985,-74.933689,11.0,NaN,US,US-PA,Bensalem,no,NaN,NaN,K00A,00A,https://www.penndot.pa.gov/TravelInPA/airports...,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,NaN,NaN,00AA,00AA,NaN,NaN,NaN


In [7]:
df_airportUbicat.columns

Index(['id', 'ident', 'type', 'name', 'latitude_deg', 'longitude_deg',
       'elevation_ft', 'continent', 'iso_country', 'iso_region',
       'municipality', 'scheduled_service', 'icao_code', 'iata_code',
       'gps_code', 'local_code', 'home_link', 'wikipedia_link', 'keywords'],
      dtype='object')

### Dataset Aerolineas usadas pivotVueloDL

In [8]:
# Carga las hojas
pivotVueloDL_h1 = pd.read_excel('pivotVueloDL.xlsx', sheet_name='Compañía - Origen')
pivotVueloDL_h2 = pd.read_excel('pivotVueloDL.xlsx', sheet_name='Compañía - Destino')

## 1.2 Limpieza básica de Nulos

### Limpieza de df_stationMeteo

In [9]:
## Guarda las variables que se ocupan en un nuevo df
## borra filas con algun nan
df_stationMeteo = df_stationMeteo[['WBAN', 'STATION NAME', 'LAT', 'LON']].dropna()

## Sustituye nombre de columnas
df_stationMeteo.columns = ['STATION_ID', 'STATION_NAME', 'LAT', 'LON']

### Limpieza de df_airportUbicat

In [10]:
## Guarda las variables que se ocupan en un nuevo df
## borra filas con algun nan
df_airportUbicat = df_airportUbicat[['iata_code', 'latitude_deg', 'longitude_deg']].dropna()

## Sustituye nombre de columnas
df_airportUbicat.columns  = ['IATA', 'LAT', 'LON']

# 2. Asignación de estaciones meteorológicas a aeropuertos

## 2.1 Cálculo de distancias

### Función que calcula distancias (haversine)

Calcula la distancia más corta entre 2 puntos en la tierra

In [11]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en km

    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c

### Busca el más cercano (estación-aeropuerto)

In [12]:
results = []

for _, aero in df_airportUbicat.iterrows():
    distances = haversine(
        aero['LAT'], aero['LON'],
        df_stationMeteo['LAT'].values,
        df_stationMeteo['LON'].values
    )

    idx_min = distances.argmin()

    results.append({
    'IATA': aero['IATA'],
    'STATION_ID': df_stationMeteo.iloc[idx_min]['STATION_ID'],
    'STATION_NAME': df_stationMeteo.iloc[idx_min]['STATION_NAME'],
    'DIST_KM': distances[idx_min]
})

# 3. Construcción del mapeo aeropuerto–estación

## 3.1 Estación final para cada aeropuerto

In [13]:
airport_station_mapping = pd.DataFrame(results)
airport_station_mapping.head(2)

,IATA,STATION_ID,STATION_NAME,DIST_KM
0,UTK,99999,UTIRIK ATOLL MARSHALL IS.,1.208882
1,OCA,99999,HIGHWAY CREEK,19.507756


In [14]:
airport_station_mapping.shape

(9060, 4)

## 3.2 Reconoce de pivotVueloDL los aeropuertos usados

In [15]:
# Obtiene los valores únicos de cada columna y los convierte en "sets" (conjuntos)
# Los sets son perfectos para comparar porque eliminan duplicados automáticamente
origenes = set(pivotVueloDL_h1['ORIGIN'].unique())
destinos = set(pivotVueloDL_h2['DEST'].unique())

# Compara las listas
solo_en_origen = origenes - destinos
solo_en_destino = destinos - origenes
estan_en_ambos = origenes.intersection(destinos)

# Muestra resultados
print(f"Valores únicos solo en ORIGIN: {solo_en_origen}")
print(f"Valores únicos solo en DEST: {solo_en_destino}")
print(f'Cantidad de aeropuertos usados {len(estan_en_ambos)}')

Valores únicos solo en ORIGIN: set()
Valores únicos solo en DEST: set()
Cantidad de aeropuertos usados 352


In [16]:
aeropuertos_used = estan_en_ambos.copy()

In [17]:
airport_station_mapping = airport_station_mapping[
    airport_station_mapping['IATA'].isin(aeropuertos_used)]

airport_station_mapping.head(2)

,IATA,STATION_ID,STATION_NAME,DIST_KM
2683,ABE,14737,LEHIGH VALLEY INTERNATIONAL AIRPORT,0.481182
2684,ABI,13962,ABILENE REGIONAL AIRPORT,0.034724


In [18]:
len(airport_station_mapping.STATION_NAME.unique())

352

## 3.3 Guarda el mapeo de los aeropuertos con su respectiva estación

In [19]:
airport_station_mapping.to_csv("airport_station_mapping.csv", index=False)

# 4. Exportación de IDs de estaciones

In [20]:
airport_station_mapping['STATION_WBAN'] = (
    'WBAN:' + airport_station_mapping['STATION_ID'].astype(str)
)

airport_station_mapping[['STATION_ID', 'STATION_WBAN']].drop_duplicates().to_csv(
    'stations_id.csv',
    index=False
)

In [21]:
stations_id = pd.read_csv('stations_id.csv')

In [22]:
stations_id.head(2)

,STATION_ID,STATION_WBAN
0,14737,WBAN:14737
1,13962,WBAN:13962
